### LAB02 - AI Plugins in Semantic Kernel

In this lab, we will learn how to use Semantic Kernel to build native and semantic functions and group them together as AI Plugins. We'll work on the following steps:

1) What are AI Plugins? 
2) Creating and importing Semantic Functions from a Plugin.
2) Creating Native Functions.
....
3) Transforming your functions in AI Plugins

#### What are AI Plugins?
As the Copilot concept evolves, it's important to think on ways to extend their capabilities, allowing them to retrieve external information, interact with external services and execute actions on the behavior of the user on a safer way, following Responsible AI principles.

The concept of **Plugins** is an answer to this, as they can act as a "bridge" between Copilots / AI Apps and the digital world. 
![Plugins Overview](images/plugins_overview.png)

With Plugins, it's possible to expand Copilots capabilities and promote interoperability across the industry as Semantic Kernel adopts an open standard for plugins, the [OpenAPI Specification](https://swagger.io/specification/).

Let's get started with the creation of Functions, which will be the base of our AI Plugins. Here, we'll explore Plugins as a collection of functions. 

#### Creating and importing Semantic Functions

In the first lesson, we have created and used an [**inline**](https://learn.microsoft.com/en-us/semantic-kernel/ai-orchestration/plugins/semantic-functions/inline-semantic-functions?tabs=python) Semantic Function to categorize a call.However this approach is fine for development and testing, we need a more robust way to ensure we can reuse the functions across projects and rapidly maintain / change the prompts and function parameters as required.

Therefore, we'll now explore how we can reuse such functions. Semantic Kernel allows us to import files to define functions. We must follow a standard folder structure when creating the files. Here's an example:

```
📁 plugins
│
└─── 📂 plugin_name_A
     |
     └─── 📂 function_name_A 
     |      |
     |      └───📄 skprompt.txt
     |      └───📄 config.json
     |
     └─── 📂 function_name_B 
            |
            └───📄 skprompt.txt
            └───📄 config.json
    📂 plugin_name_B
     |
     └─── 📂 function_name_C 
            |
            └───📄 skprompt.txt
            └───📄 config.json
```

With this structure in mind, we already start organizing our plugins and grouping together functions that belong to a plugin. For each Function (represented by a Folder) we must have two files:
* **skprompt.txt**: it's the file that will contain the prompt of a Semantic Function. Therefore, we can easily maintain our functions by reviewing the prompts;
* **config.json**: it's a configuration file that will describe the function in natural language, define which parameters the function requires, and the overall configuration the LLM should use run that function, such as temperature and maximum number of tokens. 

With this structure in mind, we already defined a folder structure to contain the funcions for a Call Center solution plugin. In this lesson, we'll recreate the function used before to fit into this structure.

Copy and paste the prompt below in the skprompt.txt file inside the Semantic Function "categorize" under "callcenter" plugin:

```txt
You help a Telecom company to classify problems reported by their subscribers. Your role is to provide accurate classification based on problems description and a list of categories that the Telecom company uses.
Classify the problem in one of the provided categories. Only write the output category with no extra text.
Only write one category per problem description.

Examples: 
Problem: My 5G is not working well when I'm in my car.
Categories: Fixed Internet/Wifi, Mobile Internet.
Output Category: Mobile Internet

Problem: My TV is not streaming well from Youtube. It seems that the wifi in my bedroom is weak.
Categories: Mobile Internet, TV, Fixed Internet/Wifi.
Output Category: Fixed Internet/Wifi

Problem: It's impossible to work today, my internet here in my home is so slow!
Categories: Landiline, Mobile Internet, TV, Fixed Internet/Wifi.
Output Category: Fixed Internet/Wifi

Problem: {{$problem}}
Categories: {{$categories}}
Output Category: 
```

Next, modify the config.json file to ensure it looks like these:
```json
{
     "schema": 1,
     "type": "completion",
     "description": "Categorize a problem reported in a call to the Telecom company based on provided categories.",
     "completion": {
          "max_tokens": 10,
          "temperature": 0.5,
          "top_p": 0.0,
          "presence_penalty": 0.0,
          "frequency_penalty": 0.0
     },
     "input": {
          "parameters": [
               {
                    "name": "problem",
                    "description": "The problem reported by the user that needs to be classified.",
                    "defaultValue": ""
               },
               {
                    "name": "categories",
                    "description": "A set of categories used by the Telecom company to classify the problems.",
                    "defaultValue": ""
               }
          ]
     }
}
```

Your folder structure should look like this. Let's define a variable to represent our Plugins folder.
```
📁 plugins
│
└─── 📂 callcenter
     |
     └─── 📂 categorize 
     |      |
     |      └───📄 skprompt.txt
     |      └───📄 config.json
```

In [ ]:
#Define where the plugins are stored
plugins_directory = "../plugins"

To manipulate functions, we need to ensure we have all our pre-requirements set, as we did in the previous lesson. 

In [ ]:
import semantic_kernel as sk

#Importing the Azure Text Completion service connector
from semantic_kernel.connectors.ai.open_ai import AzureTextCompletion

# Initialize the kernel
kernel = sk.Kernel()

#Read the model, API key and endpoint from the .env file
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env() 

#Since our function is a Text Completion, we need to add it to the kernel
kernel.add_text_completion_service("Text Completion - Text-DaVinci-003", 
                                   AzureTextCompletion(deployment, endpoint, api_key))

Now we can refer to our Plugin and import it to be able to use its functions.

In [ ]:
# Importing the callcenter Plugin from the plugins directory, so we can start using its functions 
callcenter_plugin = kernel.import_semantic_skill_from_directory(
    plugins_directory, "callcenter"
)

In both **config.json** and **skprompt.txt** files we have defined input parameters that "**Categorize**" function requires. In Semantic Kernel we need to use **ContextVariables** to bind values with such parameters. 

After defining the input variables, we can finally call our semantic function available in our Plugin to perform the expected operation. Please note in the code that we can have multiple functions in a Plugin, so in this case we had to explicitly mention which semantic function we would like to use.

In [10]:
#Defining the input variables for the function
variables = sk.ContextVariables()
variables["problem"] = "My 3G is terrible slow ."
variables["categories"] = "Landline, Mobile Internet, TV, Fixed Internet/Wif"

#Call the plugin with input variables
result = await kernel.run_async(
    callcenter_plugin["categorize"],
    input_vars = variables
    )

print(result)

 Mobile Internet
